In [2]:
# !pip install faiss-cpu

In [4]:
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

# import WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader

In [5]:
load_dotenv()

## load the GROQ And OpenAI API KEY 
groq_api_key=os.getenv('GROQ_API_KEY')

In [9]:
loader = PyPDFDirectoryLoader("./documents")
docs = loader.load()
print(f"Loaded {len(docs)} documents")

Loaded 12 documents


In [9]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) ## Chunk Creation
final_documents=text_splitter.split_documents(docs) #splitting
print(f"Split into {len(final_documents)} chunks")

Split into 58 chunks


In [14]:
embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'} # Use 'cuda' if you have a GPU
    )
vectors=FAISS.from_documents(final_documents, embeddings) #vector OpenAI embeddings

In [11]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided context."
              "If you don't know the answer, say 'I don't know'.\n\n"
              "Context:\n{context}"),
    ("human", "{input}"),
])

In [12]:
document_chain=create_stuff_documents_chain(llm, prompt)
retriever=vectors.as_retriever(search_kwargs={"k": 5})
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [13]:
user_question = "What is the content of the penalty rule?"
response=retrieval_chain.invoke({'input':user_question})
print("\n--- Answer ---")
print(response["answer"])


--- Answer ---
According to the context, the penalty rule was developed in relation to the specific situation of enforcement of penal bonds, and its application was later extended to other contracts. The content of the penalty rule is not explicitly stated, but it is mentioned that it requires a breach of contract for the clause to be enforceable. Additionally, it is mentioned that the penalty rule is confined to the sphere of secondary obligations, without interfering with the primary obligations between the contracting parties.
